In [38]:
import pandas as pd
import numpy as np
import os
import random
import pyarrow.parquet as pq
from rectools.dataset import Dataset
from implicit.nearest_neighbours import TFIDFRecommender, CosineRecommender, BM25Recommender
from rectools.models import ImplicitItemKNNWrapperModel, PopularModel, PureSVDModel, PopularInCategoryModel
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import numpy as np
from datetime import datetime


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)

In [2]:
def reduce_mem_usage(df: pd.DataFrame) -> pd.DataFrame:
    """Проходит по всем столбцам DataFrame и изменяет тип данных
    для уменьшения использования памяти.
    """
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype.name

        # Пропускаем столбцы с типом 'object', 'category', 'datetime'
        if col_type in ['object', 'category']:
            continue  # Пропускаем эти столбцы

        if col_type.startswith('datetime'):
            continue  # Пропускаем столбцы с временными метками

        c_min = df[col].min()
        c_max = df[col].max()

        if str(col_type)[:3] == 'int':
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                df[col] = df[col].astype(np.int64)
        else:  # Для столбцов с вещественными типами
            if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

# LLM модел

In [3]:
# model = SentenceTransformer('cointegrated/rubert-tiny') # nreimers/albert-small-v2, cointegrated/rubert-tiny, cointegrated/rubert-tiny2, ai-forever/sbert_tiny_nlu_ru

# video_stat

In [4]:
video_stat = pd.DataFrame()

parquet_file = pq.ParquetFile('video_stat.parquet')
for i, batch in tqdm(enumerate(parquet_file.iter_batches(batch_size=100000)), total=parquet_file.num_row_groups):
    d = batch.to_pandas()
    video_stat = reduce_mem_usage(pd.concat([video_stat, d], ignore_index=True))
    # if i == 8:
    #     break
video_stat.drop(columns='row_number', inplace=True)
# video_stat['category_id'] = video_stat['category_id'].astype('category')
# video_stat['category_id'] = video_stat['category_id'].cat.codes
video_stat = reduce_mem_usage(video_stat)

 50%|█████     | 1/2 [00:00<00:00,  1.07it/s]

Memory usage of dataframe is 31.47 MB
Memory usage after optimization is: 15.07 MB
Decreased by 52.1%


100%|██████████| 2/2 [00:01<00:00,  1.45it/s]

Memory usage of dataframe is 63.32 MB
Memory usage after optimization is: 30.52 MB
Decreased by 51.8%


3it [00:02,  1.58it/s]                       

...

15it [00:12,  1.01s/it]

Memory usage after optimization is: 231.74 MB
Decreased by 51.2%
Memory usage of dataframe is 492.56 MB


16it [00:13,  1.20it/s]

Memory usage after optimization is: 240.35 MB
Decreased by 51.2%


Memory usage of dataframe is 237.38 MB
Memory usage after optimization is: 237.38 MB
Decreased by 0.0%


In [5]:
video_stat

video_id            v_pub_datetime  \
0        973f24c0-15b2-434f-8740-4e2726f79c30 2024-06-15 22:58:03+03:00   
1        1a3f527c-44b2-4f7e-9641-26f078edbf2e 2021-04-18 17:27:42+03:00   
2        2f563d09-45ef-4e27-8da0-daa74235a2e7 2022-11-17 16:36:43+03:00   
3        154fac0e-1794-4406-afa0-61f688e3a764 2010-01-01 00:00:01+03:00   
4        470b4e3e-e06d-4370-80dc-34d6a78b22db 2023-11-29 16:49:46+03:00   
...                                       ...                       ...   
1555682  6e3067be-8602-4889-b296-3dbba9e29a14 2023-07-22 17:48:30+03:00   
1555683  542bb668-c8ec-4047-80c6-d98b39d3b7ce 2010-06-17 18:38:04+03:00   
1555684  eeeb7b1e-6e46-4633-a9fb-3782256857f5 2022-11-06 09:24:55+03:00   
1555685  7ef043b7-a34c-40c7-865e-1b251b00df72 2023-11-17 01:35:34+03:00   
1555686  805bbf13-b667-4e7b-8db7-fbbb9841465d 2023-11-17 21:34:04+03:00   

         v_total_comments  v_year_views  v_month_views  v_week_views  \
0                     0.0         556.0            2.0           0.0   
1                     0.0         109.0            2.0           1.0   
2                     0.0         163.0            2.0           2.0   
3                     0.0       10657.0         5227.0        1300.0   
4                     0.0         154.0           45.0          13.0   
...                   ...           ...            ...           ...   
1555682               0.0         705.0           39.0          23.0   
1555683               0.0         109.0            2.0           0.0   
1555684               0.0         162.0           20.0           4.0   
1555685               0.0         144.0           22.0           9.0   
1555686               0.0        2510.0          218.0          74.0   

         v_day_views  v_likes  v_dislikes   v_duration  ...  \
0                0.0      0.0         0.0     7.733000  ...   
1                0.0      0.0         0.0   379.000000  ...   
2                0.0      0.0         0.0  1020.767029  ...   
3              202.0     24.0         0.0    42.025002  ...   
4                1.0      0.0         0.0   695.646973  ...   
...              ...      ...         ...          ...  ...   
1555682          7.0      0.0         0.0    92.996002  ...   
1555683          0.0      0.0         0.0    65.279999  ...   
1555684          2.0      0.0         0.0    94.134003  ...   
1555685          2.0      0.0         0.0   321.829010  ...   
1555686          2.0      0.0         0.0   102.492996  ...   

         v_frac_avg_watchtime_30_day_duration  \
0                                    0.000000   
1                                    0.106860   
2                                    0.156745   
3                                    2.909949   
4                                    0.257155   
...                                       ...   
1555682                              0.171223   
1555683                              0.934436   
1555684                              0.422802   
1555685                              0.299001   
1555686                              0.744647   

         v_category_popularity_percent_7_days  \
0                                    1.656250   
1                                    5.570312   
2                                    0.342529   
3                                    0.418457   
4                                    9.312500   
...                                       ...   
1555682                             10.703125   
1555683                              1.446289   
1555684                              8.953125   
1555685                              5.488281   
1555686                              1.311523   

         v_category_popularity_percent_30_days  v_long_views_1_days  \
0                                     1.617188                  0.0   
1                                     5.648438                  0.0   
2                                     0.285400                  0.0   
3                                     0.513672      

In [6]:
len(video_stat)

1555687

In [7]:
# video_stat['txt'] = (
#     video_stat['title'].apply(lambda x: 'Заголовок: ' + str(x) + '. ')
#     + video_stat['description'].apply(lambda x: 'Описание: ' + str(x) + '. ')
#     + video_stat['category_id'].apply(lambda x: 'Категория: ' + str(x) + '. ')
# )

In [8]:
# emd_title = []
# dt_txt = video_stat['txt'].tolist()
# for i in tqdm(range(0, len(dt_txt), 16)):
#     batch_embeddings = model.encode(dt_txt[i:i + 16])
#     emd_title.extend(batch_embeddings)
#     # print(i, i + 10)
# dt_txt.clear()
# emd = pd.DataFrame(emd_title)
# emd.columns = [str(i) + '_embending' for i in emd.columns]

In [9]:
# sum(emd.isnull().sum())

In [10]:
video_stat = video_stat.reset_index(drop=True)#.join(emd)
video_stat = video_stat.rename(columns={'video_id': 'item_id'})

video_stat.head()

item_id            v_pub_datetime  \
0  973f24c0-15b2-434f-8740-4e2726f79c30 2024-06-15 22:58:03+03:00   
1  1a3f527c-44b2-4f7e-9641-26f078edbf2e 2021-04-18 17:27:42+03:00   
2  2f563d09-45ef-4e27-8da0-daa74235a2e7 2022-11-17 16:36:43+03:00   
3  154fac0e-1794-4406-afa0-61f688e3a764 2010-01-01 00:00:01+03:00   
4  470b4e3e-e06d-4370-80dc-34d6a78b22db 2023-11-29 16:49:46+03:00   

   v_total_comments  v_year_views  v_month_views  v_week_views  v_day_views  \
0               0.0         556.0            2.0           0.0          0.0   
1               0.0         109.0            2.0           1.0          0.0   
2               0.0         163.0            2.0           2.0          0.0   
3               0.0       10657.0         5227.0        1300.0        202.0   
4               0.0         154.0           45.0          13.0          1.0   

   v_likes  v_dislikes   v_duration  ...  \
0      0.0         0.0     7.733000  ...   
1      0.0         0.0   379.000000  ...   
2      0.0         0.0  1020.767029  ...   
3     24.0         0.0    42.025002  ...   
4      0.0         0.0   695.646973  ...   

   v_frac_avg_watchtime_30_day_duration  v_category_popularity_percent_7_days  \
0                              0.000000                              1.656250   
1                              0.106860                              5.570312   
2                              0.156745                              0.342529   
3                              2.909949                              0.418457   
4                              0.257155                              9.312500   

   v_category_popularity_percent_30_days  v_long_views_1_days  \
0                               1.617188                  0.0   
1                               5.648438                  0.0   
2                               0.285400                  0.0   
3                               0.513672                186.0   
4                               9.289062                  0.0   

   v_long_views_7_days  v_long_views_30_days  \
0                  0.0                   0.0   
1                  0.0                   0.0   
2                  0.0                   0.0   
3               1162.0                4550.0   
4                  8.0                  18.0   

                                               title  \
0                                   пережила свой яд   
1  🎓Знаете ли Вы, что  ؟    Интересные факты ! # ...   
2  ЗИМНЯЯ, ТЕПЛАЯ ШЛЯПА крючком. Как связать поля...   
3                                               найк   
4  SHACMAN. Шакман замена двухрядного подшипника ...   

                                         description  category_id  \
0                                                     Мультфильмы   
1  🎓Знаете ли Вы, что  ?  Интересные факты ! # 2🎓...  Развлечения   
2  Шляпа связана из пряжи NAKO Sport Wool в 100гр...        Хобби   
3                                     найк дрессура      Животные   
4                                                          Разное   

                              author_id  
0  79276150-fdaf-4626-9452-6f1ad975741d  
1  16278647-5912-48af-a071-582dcfd0e354  
2  9afa94aa-c2a6-43fc-a0d1-175eaf7fa194  
3  4af9b043-40f4-4288-85c6-192612dca35b  
4  841e6bd8-0015-4502-a3e9-aaa07112a585  

[5 rows x 42 columns]

# logs_df_2024-08-05

In [11]:
logs_df = pd.DataFrame()


parquet_file = pq.ParquetFile('logs_df_2024-08-05.parquet')
for i, batch in enumerate(parquet_file.iter_batches(batch_size=100000)):
    d = batch.to_pandas().drop(columns=['region', 'city']).rename(columns={'event_timestamp': 'datetime'})
    d = reduce_mem_usage(d)
    logs_df = pd.concat([logs_df, d], ignore_index=True)
logs_df = reduce_mem_usage(logs_df)


# parquet_file = pq.ParquetFile('logs_df_2024-08-06.parquet')
# for i, batch in enumerate(parquet_file.iter_batches(batch_size=100000)):
#     d = batch.to_pandas().drop(columns=['region', 'city']).rename({'event_timestamp': 'datetime'})
#     d = reduce_mem_usage(d)
#     logs_df = pd.concat([logs_df, d], ignore_index=True)
# logs_df = reduce_mem_usage(logs_df)

Memory usage of dataframe is 3.81 MB
Memory usage after optimization is: 3.43 MB
Decreased by 10.0%
Memory usage of dataframe is 3.81 MB
Memory usage after optimization is: 3.43 MB
Decreased by 10.0%
Memory usage of dataframe is 3.81 MB
Memory usage after optimization is: 3.43 MB
Decreased by 10.0%
Memory usage of dataframe is 3.81 MB
Memory usage after optimization is: 3.43 MB
Decreased by 10.0%
Memory usage of dataframe is 3.81 MB
Memory usage after optimization is: 3.43 MB
Decreased by 10.0%
Memory usage of dataframe is 3.81 MB
Memory usage after optimization is: 3.43 MB
Decreased by 10.0%
Memory usage of dataframe is 3.81 MB
Memory usage after optimization is: 3.43 MB
Decreased by 10.0%
Memory usage of dataframe is 3.81 MB
Memory usage after optimization is: 3.43 MB
Decreased by 10.0%
Memory usage of dataframe is 3.81 MB
Memory usage after optimization is: 3.43 MB
Decreased by 10.0%
Memory usage of dataframe is 3.81 MB
Memory usage after optimization is: 3.43 MB
Decreased by 10.0%


In [12]:
logs_df

datetime                               user_id  \
0        2024-08-05 12:12:32+03:00  bd1b6b07-e6a2-46ad-aa58-990bb7e89252   
1        2024-08-05 18:30:15+03:00  7db026fc-b55c-4580-bc26-e3df59ba6c7c   
2        2024-08-05 12:35:16+03:00  5c94a654-f376-4529-b463-b62b6b50e8ca   
3        2024-08-05 20:48:51+03:00  b972399a-c4b9-47fe-88fa-ccb1827cd131   
4        2024-08-05 05:19:19+03:00  807fb9e8-0aac-4dd2-836b-48797681f4ff   
...                            ...                                   ...   
32131105 2024-08-05 09:42:57+03:00  39ae1273-470d-4a49-9802-3736fea3c64f   
32131106 2024-08-05 22:11:50+03:00  81d784b7-cda3-41d4-a648-f3d2d0749a3f   
32131107 2024-08-05 21:20:48+03:00  489f7dac-367e-4ac2-9c99-f6e650dc4598   
32131108 2024-08-05 14:06:42+03:00  8bd363cf-5115-449e-9ebf-e82f83d3db40   
32131109 2024-08-05 23:09:57+03:00  a0350a6b-41d7-4d2e-964e-608b8f5c3349   

                                      video_id  watchtime  
0         0af133e1-4fbf-4e10-b9e8-19fbe1a011c3         32  
1         4f42aab2-6238-4381-bb53-36213916e912        340  
2         8b106b84-f3f5-4c27-820c-7e1546b43614         12  
3         305389f3-1c8d-4026-9eef-e2b875a35380       2460  
4         caeb71a4-f0f6-4c6a-ac79-31bdbb58fef1         87  
...                                        ...        ...  
32131105  67d2ee4f-4309-415d-90b4-a9b7328067d1        149  
32131106  0051ec7f-1264-4f7c-8679-38b96c2c7bad          0  
32131107                                  None          0  
32131108  8cbfd763-c21b-4371-a1f2-3806eaa5ab8b       1810  
32131109  a273119b-0f82-45b8-8aa4-7cf0f080d23f         79  

[32131110 rows x 4 columns]

In [13]:
logs_df = logs_df.rename(columns={'video_id': 'item_id'})

# create Dataset

In [14]:
logs_df = logs_df.merge(video_stat[['v_duration', 'item_id']])
logs_df['weight'] = logs_df['watchtime'] / logs_df['v_duration'] * 5
logs_df.replace(np.inf, np.nan)
logs_df.dropna(inplace=True)
logs_df['weight'] = logs_df['weight'].apply(lambda x: 5 if x > 5 else 0 if x < 0 else x)
logs_df['weight'] = logs_df['weight'].apply(round)
logs_df = logs_df.drop(columns=['v_duration'])

In [15]:
logs_df = logs_df.drop(columns=['watchtime'])

In [16]:
# video_stat = video_stat.drop(columns=['title', 'description', 'txt', 'category_id', 'author_id', 'v_pub_datetime'])

cols = video_stat.drop(columns=['title', 'description', 'author_id', 'v_pub_datetime']).columns.tolist()
cols.remove('item_id')


# Преобразуем video_stat в "длинный" формат (feature-value)
item_features_df = video_stat.melt(id_vars=['item_id'], value_vars=cols, 
                                   var_name='feature', value_name='value')

In [17]:
logs_df

datetime                               user_id  \
0        2024-08-05 12:12:32+03:00  bd1b6b07-e6a2-46ad-aa58-990bb7e89252   
1        2024-08-05 18:30:15+03:00  7db026fc-b55c-4580-bc26-e3df59ba6c7c   
2        2024-08-05 12:35:16+03:00  5c94a654-f376-4529-b463-b62b6b50e8ca   
3        2024-08-05 20:48:51+03:00  b972399a-c4b9-47fe-88fa-ccb1827cd131   
4        2024-08-05 05:19:19+03:00  807fb9e8-0aac-4dd2-836b-48797681f4ff   
...                            ...                                   ...   
29340328 2024-08-05 08:34:54+03:00  7f4f21b0-e155-4553-94f8-cafa0abe0917   
29340329 2024-08-05 09:42:57+03:00  39ae1273-470d-4a49-9802-3736fea3c64f   
29340330 2024-08-05 22:11:50+03:00  81d784b7-cda3-41d4-a648-f3d2d0749a3f   
29340331 2024-08-05 14:06:42+03:00  8bd363cf-5115-449e-9ebf-e82f83d3db40   
29340332 2024-08-05 23:09:57+03:00  a0350a6b-41d7-4d2e-964e-608b8f5c3349   

                                       item_id  weight  
0         0af133e1-4fbf-4e10-b9e8-19fbe1a011c3       0  
1         4f42aab2-6238-4381-bb53-36213916e912       0  
2         8b106b84-f3f5-4c27-820c-7e1546b43614       0  
3         305389f3-1c8d-4026-9eef-e2b875a35380       4  
4         caeb71a4-f0f6-4c6a-ac79-31bdbb58fef1       1  
...                                        ...     ...  
29340328  c0432869-f7c3-4417-95c2-8c95e413994d       4  
29340329  67d2ee4f-4309-415d-90b4-a9b7328067d1       0  
29340330  0051ec7f-1264-4f7c-8679-38b96c2c7bad       0  
29340331  8cbfd763-c21b-4371-a1f2-3806eaa5ab8b       4  
29340332  a273119b-0f82-45b8-8aa4-7cf0f080d23f       2  

[29190819 rows x 4 columns]

In [18]:
video_stat

item_id            v_pub_datetime  \
0        973f24c0-15b2-434f-8740-4e2726f79c30 2024-06-15 22:58:03+03:00   
1        1a3f527c-44b2-4f7e-9641-26f078edbf2e 2021-04-18 17:27:42+03:00   
2        2f563d09-45ef-4e27-8da0-daa74235a2e7 2022-11-17 16:36:43+03:00   
3        154fac0e-1794-4406-afa0-61f688e3a764 2010-01-01 00:00:01+03:00   
4        470b4e3e-e06d-4370-80dc-34d6a78b22db 2023-11-29 16:49:46+03:00   
...                                       ...                       ...   
1555682  6e3067be-8602-4889-b296-3dbba9e29a14 2023-07-22 17:48:30+03:00   
1555683  542bb668-c8ec-4047-80c6-d98b39d3b7ce 2010-06-17 18:38:04+03:00   
1555684  eeeb7b1e-6e46-4633-a9fb-3782256857f5 2022-11-06 09:24:55+03:00   
1555685  7ef043b7-a34c-40c7-865e-1b251b00df72 2023-11-17 01:35:34+03:00   
1555686  805bbf13-b667-4e7b-8db7-fbbb9841465d 2023-11-17 21:34:04+03:00   

         v_total_comments  v_year_views  v_month_views  v_week_views  \
0                     0.0         556.0            2.0           0.0   
1                     0.0         109.0            2.0           1.0   
2                     0.0         163.0            2.0           2.0   
3                     0.0       10657.0         5227.0        1300.0   
4                     0.0         154.0           45.0          13.0   
...                   ...           ...            ...           ...   
1555682               0.0         705.0           39.0          23.0   
1555683               0.0         109.0            2.0           0.0   
1555684               0.0         162.0           20.0           4.0   
1555685               0.0         144.0           22.0           9.0   
1555686               0.0        2510.0          218.0          74.0   

         v_day_views  v_likes  v_dislikes   v_duration  ...  \
0                0.0      0.0         0.0     7.733000  ...   
1                0.0      0.0         0.0   379.000000  ...   
2                0.0      0.0         0.0  1020.767029  ...   
3              202.0     24.0         0.0    42.025002  ...   
4                1.0      0.0         0.0   695.646973  ...   
...              ...      ...         ...          ...  ...   
1555682          7.0      0.0         0.0    92.996002  ...   
1555683          0.0      0.0         0.0    65.279999  ...   
1555684          2.0      0.0         0.0    94.134003  ...   
1555685          2.0      0.0         0.0   321.829010  ...   
1555686          2.0      0.0         0.0   102.492996  ...   

         v_frac_avg_watchtime_30_day_duration  \
0                                    0.000000   
1                                    0.106860   
2                                    0.156745   
3                                    2.909949   
4                                    0.257155   
...                                       ...   
1555682                              0.171223   
1555683                              0.934436   
1555684                              0.422802   
1555685                              0.299001   
1555686                              0.744647   

         v_category_popularity_percent_7_days  \
0                                    1.656250   
1                                    5.570312   
2                                    0.342529   
3                                    0.418457   
4                                    9.312500   
...                                       ...   
1555682                             10.703125   
1555683                              1.446289   
1555684                              8.953125   
1555685                              5.488281   
1555686                              1.311523   

         v_category_popularity_percent_30_days  v_long_views_1_days  \
0                                     1.617188                  0.0   
1                                     5.648438                  0.0   
2                                     0.285400                  0.0   
3                                     0.513672       

In [19]:
item_features_df

item_id           feature      value
0         973f24c0-15b2-434f-8740-4e2726f79c30  v_total_comments        0.0
1         1a3f527c-44b2-4f7e-9641-26f078edbf2e  v_total_comments        0.0
2         2f563d09-45ef-4e27-8da0-daa74235a2e7  v_total_comments        0.0
3         154fac0e-1794-4406-afa0-61f688e3a764  v_total_comments        0.0
4         470b4e3e-e06d-4370-80dc-34d6a78b22db  v_total_comments        0.0
...                                        ...               ...        ...
57560414  6e3067be-8602-4889-b296-3dbba9e29a14       category_id    Сериалы
57560415  542bb668-c8ec-4047-80c6-d98b39d3b7ce       category_id      Спорт
57560416  eeeb7b1e-6e46-4633-a9fb-3782256857f5       category_id  Лайфстайл
57560417  7ef043b7-a34c-40c7-865e-1b251b00df72       category_id     Разное
57560418  805bbf13-b667-4e7b-8db7-fbbb9841465d       category_id     Разное

[57560419 rows x 3 columns]

In [20]:
item_features_df.dropna(inplace=True)

In [21]:
item_features_df = reduce_mem_usage(item_features_df)

Memory usage of dataframe is 1317.45 MB
Memory usage after optimization is: 1317.45 MB
Decreased by 0.0%


In [22]:
item_features_df

item_id           feature      value
0         973f24c0-15b2-434f-8740-4e2726f79c30  v_total_comments        0.0
1         1a3f527c-44b2-4f7e-9641-26f078edbf2e  v_total_comments        0.0
2         2f563d09-45ef-4e27-8da0-daa74235a2e7  v_total_comments        0.0
3         154fac0e-1794-4406-afa0-61f688e3a764  v_total_comments        0.0
4         470b4e3e-e06d-4370-80dc-34d6a78b22db  v_total_comments        0.0
...                                        ...               ...        ...
57560414  6e3067be-8602-4889-b296-3dbba9e29a14       category_id    Сериалы
57560415  542bb668-c8ec-4047-80c6-d98b39d3b7ce       category_id      Спорт
57560416  eeeb7b1e-6e46-4633-a9fb-3782256857f5       category_id  Лайфстайл
57560417  7ef043b7-a34c-40c7-865e-1b251b00df72       category_id     Разное
57560418  805bbf13-b667-4e7b-8db7-fbbb9841465d       category_id     Разное

[57560419 rows x 3 columns]

In [24]:
dataset = Dataset.construct(
    interactions_df=logs_df,
    item_features_df=item_features_df,
    cat_item_features=['category_id']
)

# model

In [59]:
model = PopularModel()
model.fit(dataset)

In [60]:
model_knn = ImplicitItemKNNWrapperModel(CosineRecommender(K=32))
model_knn.fit(dataset)

c:\Users\Egor\AppData\Local\Programs\Python\Python311\Lib\site-packages\implicit\nearest_neighbours.py:233: RuntimeWarning: invalid value encountered in divide
  X.data = X.data / sqrt(bincount(X.row, X.data**2))[X.row]


In [61]:
model_svd = PureSVDModel(random_state=seed)
model_svd.fit(dataset)

# test predict

In [62]:
logs_df['user_id'].value_counts()

user_id
c6082ce0-f3a7-44df-a161-1d8b61730f76    91215
95661e14-061d-470f-bc47-fd382c8ad84d     6695
672e64ff-9732-4b5d-8f15-47561576f5a8     3644
ca037456-f4ed-464f-8b91-f0c83770a02b     2705
85b3981f-cf88-4a07-b42f-a9116fd301db     2446
                                        ...  
a178789d-f977-4467-91f5-cb96268129f4        1
1809a6ab-0f8f-4e7c-81c6-b0bb7240edaf        1
b2c2e412-14e1-404f-9f0e-3b4fcc35fec9        1
11a841b7-30af-4b28-a120-d3da7592363b        1
bfd7b5c7-8da6-4149-9773-69fbd9cfc76a        1
Name: count, Length: 9945547, dtype: int64

In [92]:
user = 'c6082ce0-f3a7-44df-a161-1d8b61730f76'

In [93]:
recommendations = model_knn.recommend(
    users=[user],
    dataset=dataset,
    k=10,
    filter_viewed=True
)
recommendations

user_id                               item_id  \
0  c6082ce0-f3a7-44df-a161-1d8b61730f76  d31d54e1-bea9-402d-9c05-abbb4727bddb   
1  c6082ce0-f3a7-44df-a161-1d8b61730f76  15f0f30f-a574-4bc6-b3d2-39b2973b37b4   
2  c6082ce0-f3a7-44df-a161-1d8b61730f76  91ef68e3-e289-4ded-b841-3495e9ee6fae   
3  c6082ce0-f3a7-44df-a161-1d8b61730f76  f0024987-9e2c-4bb3-a458-fa55010537d0   
4  c6082ce0-f3a7-44df-a161-1d8b61730f76  1d1eafb9-440b-4a5f-8336-0e7b2ad73da2   
5  c6082ce0-f3a7-44df-a161-1d8b61730f76  70c0b916-6ae1-4e3d-8068-1f0519ea5d23   
6  c6082ce0-f3a7-44df-a161-1d8b61730f76  b5000654-ca23-4ae9-807c-3b3286f754ae   
7  c6082ce0-f3a7-44df-a161-1d8b61730f76  cf5d4f04-b551-4688-9546-709901868171   
8  c6082ce0-f3a7-44df-a161-1d8b61730f76  4528acb4-dfd1-40e8-9fc8-3b7f80473462   
9  c6082ce0-f3a7-44df-a161-1d8b61730f76  b4de19f9-2d8d-412f-ae9b-5b333e0ced81   

      score  rank  
0       NaN     1  
1       NaN     2  
2       NaN     3  
3       NaN     4  
4  2.115735     5  
5       NaN     6  
6       NaN     7  
7  3.393111     8  
8       NaN     9  
9       NaN    10

In [94]:
video_stat[video_stat['item_id'].isin(recommendations['item_id'])][['item_id', 'title', 'category_id', 'description']]

item_id  \
135125   4528acb4-dfd1-40e8-9fc8-3b7f80473462   
430189   b4de19f9-2d8d-412f-ae9b-5b333e0ced81   
484758   1d1eafb9-440b-4a5f-8336-0e7b2ad73da2   
700979   91ef68e3-e289-4ded-b841-3495e9ee6fae   
1148277  cf5d4f04-b551-4688-9546-709901868171   
1149194  b5000654-ca23-4ae9-807c-3b3286f754ae   
1197419  70c0b916-6ae1-4e3d-8068-1f0519ea5d23   
1255246  d31d54e1-bea9-402d-9c05-abbb4727bddb   
1333067  15f0f30f-a574-4bc6-b3d2-39b2973b37b4   
1530860  f0024987-9e2c-4bb3-a458-fa55010537d0   

                                                     title  category_id  \
135125       NextRP┃Встретили Генерала ФСБ вернулись в ДПС    Видеоигры   
430189                  NextRP┃Мы  в опг что будет дальше?    Видеоигры   
484758                           «Одинокий волк». 24 серия      Сериалы   
700979   СТИНТ СМОТРИТ: РАДИКАЛЬНЫЕ ВЕГАНЫ В ТИКТОКЕ //...  Развлечения   
1148277                                 H 3x09 patlefourbe      Сериалы   
1149194             Гта 5 доставляем подделанные документы    Видеоигры   
1197419               Чип и Дейл Все боссы из всех частей.    Видеоигры   
1255246              Летний отдых с шашлычками с друзьями!      UNKNOWN   
1333067  Авто из Кореи во Владивостоке без пробега - Об...       Разное   
1530860  Отзыв Владельца Ивеко Дейли III,600т.км, 18 ле...       Разное   

                                               description  
135125       ⚡Скачивай NEXTRP: 💰Вводи мой промокод: VLADIK  
430189   ⚡Скачивай NEXTRP: 💰Вводи мой промокод: VLADIK ...  
484758   Помощник Гирина Носков сообщает, что Яну увезл...  
700979   ❗️ Данный видеоролик не имеет цели оскорбить к...  
1148277                                 H 3x09 patlefourbe  
1149194                                                     
1197419          Битвы со всеми боссами из данной дилогии.  
1255246  #летникrutube #лето #summer #шашлык #друзья #о...  
1333067  Авто из Кореи, Доставка Автовозом до вашего го...  
1530860  Отзыв владельца Ивеко Дейли 2002 г.в., Iveco D...

In [95]:
logs_df[(logs_df['user_id'] == user) & (logs_df['item_id'].isin(recommendations['item_id']))]

Empty DataFrame
Columns: [datetime, user_id, item_id, weight]
Index: []

In [96]:
recommendations = model_svd.recommend(
    users=[user],
    dataset=dataset,
    k=10,
    filter_viewed=True
)
recommendations

user_id                               item_id  \
0  c6082ce0-f3a7-44df-a161-1d8b61730f76  84b58977-7727-476b-9df5-a75d2db23088   
1  c6082ce0-f3a7-44df-a161-1d8b61730f76  cf4ccc65-7d45-4834-a9b8-3d2664d043f6   
2  c6082ce0-f3a7-44df-a161-1d8b61730f76  b26e54b4-dbc1-408f-831d-1f2986678218   
3  c6082ce0-f3a7-44df-a161-1d8b61730f76  46fbd9a1-8914-475a-a7a5-f4557b92910d   
4  c6082ce0-f3a7-44df-a161-1d8b61730f76  989976b7-3afd-48af-9797-970b0c9e53a5   
5  c6082ce0-f3a7-44df-a161-1d8b61730f76  5179f84e-673c-4daa-ba8c-fe01af1f8d7d   
6  c6082ce0-f3a7-44df-a161-1d8b61730f76  7be32c89-f77b-484e-ad1d-06d2061319af   
7  c6082ce0-f3a7-44df-a161-1d8b61730f76  cacbfd88-d476-4fbb-bf2e-24ed48f8041e   
8  c6082ce0-f3a7-44df-a161-1d8b61730f76  7b3229a3-fc4f-4bf2-a7d5-ff4c0dc429d1   
9  c6082ce0-f3a7-44df-a161-1d8b61730f76  4a524fbc-23ec-42c0-a291-7eac1fce7206   

      score  rank  
0  0.020852     1  
1  0.019364     2  
2  0.019249     3  
3  0.018268     4  
4  0.018109     5  
5  0.014865     6  
6  0.013304     7  
7  0.011986     8  
8  0.011835     9  
9  0.011713    10

In [97]:
video_stat[video_stat['item_id'].isin(recommendations['item_id'])][['item_id', 'title', 'category_id', 'description']]

item_id  \
75       46fbd9a1-8914-475a-a7a5-f4557b92910d   
149      989976b7-3afd-48af-9797-970b0c9e53a5   
504464   cacbfd88-d476-4fbb-bf2e-24ed48f8041e   
827112   cf4ccc65-7d45-4834-a9b8-3d2664d043f6   
916631   7b3229a3-fc4f-4bf2-a7d5-ff4c0dc429d1   
942649   4a524fbc-23ec-42c0-a291-7eac1fce7206   
1100227  84b58977-7727-476b-9df5-a75d2db23088   
1256004  b26e54b4-dbc1-408f-831d-1f2986678218   
1480836  5179f84e-673c-4daa-ba8c-fe01af1f8d7d   
1524289  7be32c89-f77b-484e-ad1d-06d2061319af   

                                                     title  category_id  \
75           Бурый медвежонок знакомится с морем. 33 серия        Детям   
149      Игрушечная страна: Господин гоблин Табби. 21 с...        Детям   
504464   Сказка 3 медведя: новая сказка 3D  Захватывающ...        Детям   
827112           Маша и Медведь, Круги на траве, 124 серия  Мультфильмы   
916631                Красная Шапочка новая сказка #сказка  Развлечения   
942649        Маша и Медведь, Лимонная лихорадка,132 серия  Мультфильмы   
1100227       Батут. Про Миру и Гошу. Мультфильм для детей  Мультфильмы   
1256004  Сказка про коровку: мультфильм о дружбе #сказк...        Детям   
1480836  Сказка: У лукоморья. Автор А. С. Пушкин – школ...        Детям   
1524289         Маша и Медведь, Чудеса медицины, 122 серия  Мультфильмы   

                                               description  
75                                        Бурый медвежонок  
149                                  Господин гоблин Табби  
504464   По вопросам сотрудничества, создания коммерчес...  
827112                                                      
916631   По вопросам сотрудничества, создания коммерчес...  
942649                                                      
1100227  Бейби очень хочет помогать! Гоше надо накачать...  
1256004  По вопросам сотрудничества, создания коммерчес...  
1480836  По вопросам сотрудничества, создания коммерчес...  
1524289

In [98]:
logs_df[(logs_df['user_id'] == user) & (logs_df['item_id'].isin(recommendations['item_id']))]

Empty DataFrame
Columns: [datetime, user_id, item_id, weight]
Index: []

In [99]:
recommendations = model.recommend(
    users=[user],
    dataset=dataset,
    k=10,
    filter_viewed=True
)
recommendations

user_id                               item_id  \
0  c6082ce0-f3a7-44df-a161-1d8b61730f76  85372f69-97fc-485d-ae4a-91e36aeb4db3   
1  c6082ce0-f3a7-44df-a161-1d8b61730f76  70c1cf85-d20b-412a-b47d-5eeaec7c5a59   
2  c6082ce0-f3a7-44df-a161-1d8b61730f76  e22762d9-d61c-4b8b-90e4-892505ac4b56   
3  c6082ce0-f3a7-44df-a161-1d8b61730f76  8c640152-ce3a-47a6-833b-ac70701e4734   
4  c6082ce0-f3a7-44df-a161-1d8b61730f76  af29ad14-35f6-4ccb-af8b-89a0ecf5f4c6   
5  c6082ce0-f3a7-44df-a161-1d8b61730f76  f0086c15-3718-411b-ad40-bc1fb67f67cb   
6  c6082ce0-f3a7-44df-a161-1d8b61730f76  264a9f93-c1cd-4023-bbac-a5ea8a965f0c   
7  c6082ce0-f3a7-44df-a161-1d8b61730f76  265cb8a0-6c8b-4256-a55d-e102c239886e   
8  c6082ce0-f3a7-44df-a161-1d8b61730f76  6e0efb88-e04c-4244-8e5d-880c87a1d74b   
9  c6082ce0-f3a7-44df-a161-1d8b61730f76  e71fbe48-34a7-462c-af6a-2e44571de12e   

      score  rank  
0  647287.0     1  
1  240739.0     2  
2  213450.0     3  
3  120607.0     4  
4  115482.0     5  
5  111643.0     6  
6  105485.0     7  
7  103736.0     8  
8   99275.0     9  
9   92520.0    10

In [100]:
recommendations

user_id                               item_id  \
0  c6082ce0-f3a7-44df-a161-1d8b61730f76  85372f69-97fc-485d-ae4a-91e36aeb4db3   
1  c6082ce0-f3a7-44df-a161-1d8b61730f76  70c1cf85-d20b-412a-b47d-5eeaec7c5a59   
2  c6082ce0-f3a7-44df-a161-1d8b61730f76  e22762d9-d61c-4b8b-90e4-892505ac4b56   
3  c6082ce0-f3a7-44df-a161-1d8b61730f76  8c640152-ce3a-47a6-833b-ac70701e4734   
4  c6082ce0-f3a7-44df-a161-1d8b61730f76  af29ad14-35f6-4ccb-af8b-89a0ecf5f4c6   
5  c6082ce0-f3a7-44df-a161-1d8b61730f76  f0086c15-3718-411b-ad40-bc1fb67f67cb   
6  c6082ce0-f3a7-44df-a161-1d8b61730f76  264a9f93-c1cd-4023-bbac-a5ea8a965f0c   
7  c6082ce0-f3a7-44df-a161-1d8b61730f76  265cb8a0-6c8b-4256-a55d-e102c239886e   
8  c6082ce0-f3a7-44df-a161-1d8b61730f76  6e0efb88-e04c-4244-8e5d-880c87a1d74b   
9  c6082ce0-f3a7-44df-a161-1d8b61730f76  e71fbe48-34a7-462c-af6a-2e44571de12e   

      score  rank  
0  647287.0     1  
1  240739.0     2  
2  213450.0     3  
3  120607.0     4  
4  115482.0     5  
5  111643.0     6  
6  105485.0     7  
7  103736.0     8  
8   99275.0     9  
9   92520.0    10

In [101]:
video_stat[video_stat['item_id'].isin(recommendations['item_id'])][['item_id', 'title', 'category_id', 'description']]

item_id  \
141599   f0086c15-3718-411b-ad40-bc1fb67f67cb   
465163   265cb8a0-6c8b-4256-a55d-e102c239886e   
534826   85372f69-97fc-485d-ae4a-91e36aeb4db3   
661206   e22762d9-d61c-4b8b-90e4-892505ac4b56   
734116   e71fbe48-34a7-462c-af6a-2e44571de12e   
829318   8c640152-ce3a-47a6-833b-ac70701e4734   
905523   6e0efb88-e04c-4244-8e5d-880c87a1d74b   
1065830  264a9f93-c1cd-4023-bbac-a5ea8a965f0c   
1133620  af29ad14-35f6-4ccb-af8b-89a0ecf5f4c6   
1228938  70c1cf85-d20b-412a-b47d-5eeaec7c5a59   

                                                     title   category_id  \
141599                       10 глупых вопросов ШОУРАННЕРУ      Интервью   
465163   Сливочный суп с рисом на кокосовом молоке с кр...           Еда   
534826               Выжить в Дубае. Возвращение, 8 выпуск  Телепередачи   
661206             Национальные блюда французской кухни ??           Еда   
734116               Выжить в Дубае. Возвращение, 7 выпуск  Телепередачи   
829318                      10 глупых вопросов 3D ИНЖЕНЕРУ      Интервью   
905523            Последний герой. Русский сезон, 6 выпуск  Телепередачи   
1065830                               Терминатор по-русски   Развлечения   
1133620                10 глупых вопросов ПРОДАВЦУ НА OZON      Интервью   
1228938                                  Титаны, 11 выпуск  Телепередачи   

                                               description  
141599   Героиня нового выпуска — шоураннер проектов «Т...  
465163   Одновременно жгучий и сливочно-мягкий, кислый ...  
534826   В новом шоу  звёздная и народная команда снова...  
661206   #франция #французскаякухня #еда #кулинария #блюда  
734116   В новом шоу  звёздная и народная команда снова...  
829318   Герой нового выпуска «10 глупых вопросов» — 3D...  
905523   Участников проекта ждёт долгожданное объединен...  
1065830  Новый комедийный сериал ТНТ с Павлом Рассомахи...  
1133620  Героиня нового выпуска «10 глупых вопросов» — ...  
1228938  Искусственный интеллект и нейросети настолько ...

In [102]:
logs_df[(logs_df['user_id'] == user) & (logs_df['item_id'].isin(recommendations['item_id']))]

Empty DataFrame
Columns: [datetime, user_id, item_id, weight]
Index: []

In [103]:
video_stat[video_stat['item_id'].isin(logs_df[logs_df['user_id'] == user]['item_id'])][['item_id', 'title', 'category_id', 'description']]

item_id  \
4666     7ec31271-2608-44f0-9b75-f798ec2c26d9   
16562    0bd2c5ad-5850-4ad0-a2cf-457c079c99ca   
55297    3afbec12-8510-4ea4-840b-f9f6c34b2d09   
84320    9ad6fa30-1610-4cdc-9528-5b256de72949   
190816   93ca9dca-cf07-49fd-9f7a-95e269a9e269   
201605   6ed19186-3a93-4a05-a58a-b6bdfdb9126a   
214661   96b537a8-810e-48f6-8260-a931a4efc7d5   
296830   af5ee2d7-53e6-4b50-8559-52e130d6c527   
333803   0d7dc4da-7894-4934-a353-51d4fd8b374c   
351490   1c26bc0d-55b3-402a-add7-f016bdb4a10d   
374526   0dcc8f6e-9045-4bf7-9ac8-97425304a0db   
427442   0ceea106-c0eb-43e5-ac30-d49a5ba89997   
450079   8e4b920c-b23e-485e-834e-5a7f99c4cfb6   
455272   18e198f4-5480-4ff6-8f58-0edf91203ce1   
457328   a62defb2-79bb-4942-9d77-163fcb09c334   
488254   14fd8ae5-c73f-4918-8c41-49b84a2931b2   
505768   a4fe4bdb-ce5e-49e8-acee-529c4508960d   
515823   dc089dc5-7c4b-44eb-b9d0-a396e535b5a3   
522810   30e00d28-584b-46a5-97bc-56b0cb3aa785   
599313   5d5880b1-171b-4b3d-8c89-8716ca368356   
665089   b63919c4-8a24-4b41-89ea-7283f2669224   
730114   c9266912-8870-4fed-9efd-a5c22a76667b   
735193   3d4e1478-438d-4585-975a-39d7a38888ec   
782844   7b3ddf1a-b2fc-45c7-80f3-8b43c0120b7f   
821831   1e5e7ebe-ec18-43d5-a2ed-2650f96c6a10   
951533   b41876b3-67c2-4331-a4c4-fb6999ace3bb   
989946   9d342b90-987f-4ce8-863d-2b91f97e6635   
1015876  a13b5d75-56ab-4a1c-8bd0-4ad31b11de0c   
1055739  b4ff33e0-6ab5-43ab-af56-602090d9c2ee   
1065050  ffe26d6b-a73a-428c-bc81-33cfb66a1e57   
1089458  a7f9db2d-0be0-41b3-af8e-94990ee8c16d   
1256335  57c0f850-0158-4ba9-8cb6-6d9093c24976   
1289421  4b75a670-aa74-4d9c-a340-6bca603f4e1e   
1295967  8077515e-783a-4c66-952b-280f1cc0b62f   
1321068  40fe6175-2dc5-444c-9d23-60354d9bafc7   
1328699  703729e4-0f3b-475a-b12a-29fea02a94bd   
1367200  6dfb2eaa-1282-472d-86e8-e75778ae40b6   
1404679  c127054d-a16a-4b5d-b344-1e43124f9ca3   
1415975  1e96b823-20dc-434a-8a31-bbd9a3ae0f5e   
1434804  40c1368a-6185-4278-aaaf-61021965344a   
1487235  a26cd82f-8a44-43ff-ad5b-2f2892b22cf2   
1490884  6c8128e9-8021-4ac5-b642-ddb2d405e752   

                                                     title  \
4666              Рекламная Мастерская РЕКВИС. ( @REKVIS )   
16562    Авто из Кореи. Какие авто выкупаем_ Обзор со с...   
55297    Как НЕ переплатить и сделать хороший ремонт: с...   
84320    ARK_ Survival Ascended, Артефакт жестокости TH...   
190816   Барби супер модель. 😍 Игра на сега мега драйв ...   
201605   Самые нашумевшие новости о знаменитостях. Фейк...   
214661   Татьяна Буланова — «Мама моя милая» (Премьера ...   
296830   КОМПЛЕКТАЦИЯ ДИЗАЙН-ПРОЕКТОВ / КАК СЭКОНОМИТЬ ...   
333803     RMRP CRIMINAL MOSCOW - СЕМЬЯ THE END #1 ⧸ GTA 5   
351490                         Электронная музыка Dj. Kygo   
374526       Переживи 100 Дней в Бункере, Выиграй $500,000   
427442                    Welcome CV - Vladimir Strelnikov   
450079    Animal Crossing ｜ Дом построен! ｜ Прохождение №4   
455272                                      Западные танки   
457328                 Макс Лидов Остров живой звук Кремль   
488254   Зачем нужен дизайнер интерьеров? / Битва дизай...   
505768   ФИЛЬМ О БРИТНИ СПИРС I АВТОБИОГРАФИЯ БРИТНИ СП...   
515823                       ＂REKVIS СПб＂ наружная реклама   
522810     RMRP CRIMINAL MOSCOW - СЕМЬЯ THE END #5 ⧸ GTA 5   
599313    Раскачал Кремль Разбуди меня Марка №1 Макс Лидов   
665089   ШОУ ХОЧУ ЗНАТЬ ЛАЙТ | РЕКЛАМА 90-х и 2000-х | ...   
730114     ОТКРЫТИЕ НАГРАДЫ ЗА БИТВУ СТРИМЕРОВ в сталкрафт   
735193          МЕТКО СТРЕЛЯЕШЬ? ПРИЗ ТВОЙ!!! DDD ЧЕЛЛЕНДЖ   
782844   Котенок и Мышь– мультфильм о дружбе. Добрая ск...   
821831   ПОЙМАЛ ЗАНОС В LIGHTNING STORM | ПОЙМАЛ ИГРУ С...   
951533                                Капкейки идут в рост   
989946                   УГАДАЙ ВКУС ИЛИ ПОЛУЧИ НАКАЗАНИЕ!   
1015876  Как найти работу после IT-курсов? Почему не б...   
1055739         Макс Лидов Сверхъестественная концерт live   
1065050                    Макс Лидов - #Птица #голос #хит   
1089458 

In [104]:
tmp_dt = video_stat[video_stat['item_id'].isin(logs_df[logs_df['user_id'] == user]['item_id'])][['item_id', 'title', 'category_id', 'description']]
sns.set(rc={'figure.figsize': (15, 8)})
colors = sns.color_palette('pastel')[0:5]
labels = sorted(tmp_dt['category_id'].unique())
dt = np.array([len(tmp_dt[tmp_dt['category_id'] == i])for i in labels])
plt.pie(dt, labels=labels, colors = colors, autopct='%.0f%%')
plt.show()

In [105]:
import json


json_req = {}
for i in video_stat[video_stat['item_id'].isin(recommendations['item_id'])][['item_id', 'title', 'category_id', 'description']].values:
    json_req[i[0]] = i[1:].tolist()


with open('json_req.json', 'w', encoding='utf-8') as js_r:
    json.dump(json_req, js_r, ensure_ascii=False, indent=4)

# save model and data base

In [107]:
import pickle

with open("PopularModel.pkl", "wb") as f:
    pickle.dump(model, f)


with open("ImplicitItemKNNWrapperModel.pkl", "wb") as f:
    pickle.dump(model_knn, f)


with open("PureSVDModel.pkl", "wb") as f:
    pickle.dump(model_svd, f)


with open("dataset.pkl", "wb") as f:
    pickle.dump(dataset, f)

# conect with global data

In [108]:
logs_df

datetime                               user_id  \
0        2024-08-05 12:12:32+03:00  bd1b6b07-e6a2-46ad-aa58-990bb7e89252   
1        2024-08-05 18:30:15+03:00  7db026fc-b55c-4580-bc26-e3df59ba6c7c   
2        2024-08-05 12:35:16+03:00  5c94a654-f376-4529-b463-b62b6b50e8ca   
3        2024-08-05 20:48:51+03:00  b972399a-c4b9-47fe-88fa-ccb1827cd131   
4        2024-08-05 05:19:19+03:00  807fb9e8-0aac-4dd2-836b-48797681f4ff   
...                            ...                                   ...   
29340328 2024-08-05 08:34:54+03:00  7f4f21b0-e155-4553-94f8-cafa0abe0917   
29340329 2024-08-05 09:42:57+03:00  39ae1273-470d-4a49-9802-3736fea3c64f   
29340330 2024-08-05 22:11:50+03:00  81d784b7-cda3-41d4-a648-f3d2d0749a3f   
29340331 2024-08-05 14:06:42+03:00  8bd363cf-5115-449e-9ebf-e82f83d3db40   
29340332 2024-08-05 23:09:57+03:00  a0350a6b-41d7-4d2e-964e-608b8f5c3349   

                                       item_id  weight  
0         0af133e1-4fbf-4e10-b9e8-19fbe1a011c3       0  
1         4f42aab2-6238-4381-bb53-36213916e912       0  
2         8b106b84-f3f5-4c27-820c-7e1546b43614       0  
3         305389f3-1c8d-4026-9eef-e2b875a35380       4  
4         caeb71a4-f0f6-4c6a-ac79-31bdbb58fef1       1  
...                                        ...     ...  
29340328  c0432869-f7c3-4417-95c2-8c95e413994d       4  
29340329  67d2ee4f-4309-415d-90b4-a9b7328067d1       0  
29340330  0051ec7f-1264-4f7c-8679-38b96c2c7bad       0  
29340331  8cbfd763-c21b-4371-a1f2-3806eaa5ab8b       4  
29340332  a273119b-0f82-45b8-8aa4-7cf0f080d23f       2  

[29190819 rows x 4 columns]

In [109]:
video_stat

item_id            v_pub_datetime  \
0        973f24c0-15b2-434f-8740-4e2726f79c30 2024-06-15 22:58:03+03:00   
1        1a3f527c-44b2-4f7e-9641-26f078edbf2e 2021-04-18 17:27:42+03:00   
2        2f563d09-45ef-4e27-8da0-daa74235a2e7 2022-11-17 16:36:43+03:00   
3        154fac0e-1794-4406-afa0-61f688e3a764 2010-01-01 00:00:01+03:00   
4        470b4e3e-e06d-4370-80dc-34d6a78b22db 2023-11-29 16:49:46+03:00   
...                                       ...                       ...   
1555682  6e3067be-8602-4889-b296-3dbba9e29a14 2023-07-22 17:48:30+03:00   
1555683  542bb668-c8ec-4047-80c6-d98b39d3b7ce 2010-06-17 18:38:04+03:00   
1555684  eeeb7b1e-6e46-4633-a9fb-3782256857f5 2022-11-06 09:24:55+03:00   
1555685  7ef043b7-a34c-40c7-865e-1b251b00df72 2023-11-17 01:35:34+03:00   
1555686  805bbf13-b667-4e7b-8db7-fbbb9841465d 2023-11-17 21:34:04+03:00   

         v_total_comments  v_year_views  v_month_views  v_week_views  \
0                     0.0         556.0            2.0           0.0   
1                     0.0         109.0            2.0           1.0   
2                     0.0         163.0            2.0           2.0   
3                     0.0       10657.0         5227.0        1300.0   
4                     0.0         154.0           45.0          13.0   
...                   ...           ...            ...           ...   
1555682               0.0         705.0           39.0          23.0   
1555683               0.0         109.0            2.0           0.0   
1555684               0.0         162.0           20.0           4.0   
1555685               0.0         144.0           22.0           9.0   
1555686               0.0        2510.0          218.0          74.0   

         v_day_views  v_likes  v_dislikes   v_duration  ...  \
0                0.0      0.0         0.0     7.733000  ...   
1                0.0      0.0         0.0   379.000000  ...   
2                0.0      0.0         0.0  1020.767029  ...   
3              202.0     24.0         0.0    42.025002  ...   
4                1.0      0.0         0.0   695.646973  ...   
...              ...      ...         ...          ...  ...   
1555682          7.0      0.0         0.0    92.996002  ...   
1555683          0.0      0.0         0.0    65.279999  ...   
1555684          2.0      0.0         0.0    94.134003  ...   
1555685          2.0      0.0         0.0   321.829010  ...   
1555686          2.0      0.0         0.0   102.492996  ...   

         v_frac_avg_watchtime_30_day_duration  \
0                                    0.000000   
1                                    0.106860   
2                                    0.156745   
3                                    2.909949   
4                                    0.257155   
...                                       ...   
1555682                              0.171223   
1555683                              0.934436   
1555684                              0.422802   
1555685                              0.299001   
1555686                              0.744647   

         v_category_popularity_percent_7_days  \
0                                    1.656250   
1                                    5.570312   
2                                    0.342529   
3                                    0.418457   
4                                    9.312500   
...                                       ...   
1555682                             10.703125   
1555683                              1.446289   
1555684                              8.953125   
1555685                              5.488281   
1555686                              1.311523   

         v_category_popularity_percent_30_days  v_long_views_1_days  \
0                                     1.617188                  0.0   
1                                     5.648438                  0.0   
2                                     0.285400                  0.0   
3                                     0.513672       